In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Datafile spec
* 710 초 ~ 720 초 길이의 sample file
* sample 당 약 39600 line == 약 720초 동안 39600번 sampling 했다는 의미
* 39600/720 = 55 -> 초당 55번 sampling 했음
* 따라서 smapling_rate = 55



In [ ]:
def _get_samples(sample_file, sample_rate=55.0, sample_sec=60.0):
  
  data = np.loadtxt(sample_file, delimiter=' ')
  
  data_len = len(data)
  sample_len = int(sample_rate * sample_sec)
  sample_cnt = data_len // sample_len

  print(sample_file)
  print(f'sample_cnt: {sample_cnt}')
  
  if sample_cnt == 0:
    return None
  
  truncate_len = int(data_len % sample_len)
  data = data[:data_len - truncate_len]

  # df = df[["GyroX", "GyroY", "GyroZ", "MagX", "MagY", "MagZ", "GraX", "GraY", "GraZ", "LinearAccX", "LinearAccY", "LinearAccZ", "Pressure", "AccX", "AccY", "AccZ"]]
  acc_norm = data[:, 13:]
  acc_norm = acc_norm ** 2
  acc_norm = np.sqrt(acc_norm.sum(axis=1))

  gyro_y = data[:, 1]

  acc_norm_samples = np.zeros((sample_cnt, sample_len,))
  gyro_y_samples = np.zeros((sample_cnt, sample_len,))

  for i in range(sample_cnt):
    acc_norm_samples[i] = acc_norm[sample_len * i : sample_len * (i + 1)]
    gyro_y_samples[i] = gyro_y[sample_len * i : sample_len * (i + 1)]
  
  return acc_norm_samples, gyro_y_samples

In [ ]:
def _make_spectrograms(samples, sample_rate, stride_ms, window_ms, eps):
  
  spectrograms = list()

  stride_size = int(0.001 * sample_rate * stride_ms)
  window_size = int(0.001 * sample_rate * window_ms)

  for sample in samples:
    # Extract strided windows
    truncate_size = int((len(sample) - window_size) % stride_size)
    sample = sample[:len(sample) - truncate_size]
    nshape = (window_size, (len(sample) - window_size) // stride_size + 1)
    nstrides = (sample.strides[0], sample.strides[0] * stride_size)
    windows = np.lib.stride_tricks.as_strided(sample, 
                                          shape = nshape, strides = nstrides)
    
    assert np.all(windows[:, 1] == sample[stride_size:(stride_size + window_size)])

    # Window weighting, squared Fast Fourier Transform (fft), scaling
    weighting = np.hanning(window_size)[:, None]
    
    fft = np.fft.rfft(windows * weighting, axis=0)
    fft = np.absolute(fft)
    fft = fft**2
    
    scale = np.sum(weighting**2) * sample_rate
    fft *= (2.0 / scale)
    fft[(0, -1), :] /= scale
    
    # Prepare fft frequency list
    freqs = float(sample_rate) / window_size * np.arange(fft.shape[0])
    
    # Compute spectrogram feature
    #ind = np.where(freqs <= max_freq)[0][-1] + 1
    #spectrogram = np.log(fft[:ind, :] + eps)
    spectrogram = np.log(fft + eps)

    spectrograms.append(spectrogram)

  return spectrograms

In [ ]:
def _make_dir(dirpath):
  dirpath_split = dirpath.split(os.path.sep)
  new_path = '/'
  for directory in dirpath_split:
    new_path = os.path.join(new_path, directory)
    if not os.path.exists(new_path):
      os.mkdir(new_path)

In [ ]:
def _get_output_filename(output_dir, filename, ext):
    n = 1
    while n < 1000:
        if n < 10:
            filename_n = f'{filename}_00{n}.{ext}' 
        elif n < 100:
            filename_n = f'{filename}_0{n}.{ext}'
        else:
            filename_n = f'{filename}_{n}.{ext}'

        output_file = os.path.join(output_dir, filename_n)

        if not os.path.exists(output_file):
            return output_file
            
        n += 1
      
    return None

In [ ]:
def generate_spectrograms(datadir, targetdir, sample_sec=60.0, sample_rate = 55.0,
                          stride_ms = 100.0, window_ms = 5000.0, eps = 1e-14):

  if not os.path.exists(targetdir):
    _make_dir(targetdir)

  for filename in os.listdir(datadir):
    filepath = os.path.join(datadir, filename)
    acc_norm_samples, gyro_y_samples = _get_samples(filepath)

    acc_spectrograms = _make_spectrograms(acc_norm_samples, sample_rate, stride_ms, window_ms, eps)
    gyro_spectrograms = _make_spectrograms(gyro_y_samples, sample_rate, stride_ms, window_ms, eps)

    for acc_spectrogram, gyro_spectrogram in zip(acc_spectrograms, gyro_spectrograms):
      fig = plt.figure(figsize=(20,15))

      acc_plot = fig.add_subplot(2,1, 1)
      gyro_plot = fig.add_subplot(2,1,2)

      acc_plot.pcolormesh(range(acc_spectrogram.shape[1]), range(acc_spectrogram.shape[0]), acc_spectrogram, cmap=parula_map, shading='auto')
      gyro_plot.pcolormesh(range(gyro_spectrogram.shape[1]), range(gyro_spectrogram.shape[0]), gyro_spectrogram, cmap=parula_map, shading='auto')

      acc_plot.axis('off')
      gyro_plot.axis('off')
      plt.subplots_adjust(hspace=0)

      plt.savefig(_get_output_filename(targetdir, filename, 'jpg'), bbox_inches='tight',pad_inches = 0, dpi=330)
      fig.clf()
      plt.close('all')

In [ ]:
datadir_base = '/content/drive/Shareddrives/Project/CNN_spectrogram_our_dataset/dataset/test'
targetdir_base = '/content/drive/Shareddrives/Project/CNN_spectrogram_our_dataset/spectrograms/test'

In [ ]:
position = 'Pocket'
modes = ['walking', 'still', 'bus', 'car', 'powerChar', 'metro']

In [ ]:
modes = os.listdir(os.path.join(datadir_base, position))
for mode in modes:
  datadir = os.path.join(datadir_base, position, mode)
  targetdir = os.path.join(targetdir_base, position, mode)
  generate_spectrograms(datadir, targetdir)

# Spectrogram을 만들기 위한 colormap

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

cm_data = [[0.2081, 0.1663, 0.5292], [0.2116238095, 0.1897809524, 0.5776761905], 
 [0.212252381, 0.2137714286, 0.6269714286], [0.2081, 0.2386, 0.6770857143], 
 [0.1959047619, 0.2644571429, 0.7279], [0.1707285714, 0.2919380952, 
  0.779247619], [0.1252714286, 0.3242428571, 0.8302714286], 
 [0.0591333333, 0.3598333333, 0.8683333333], [0.0116952381, 0.3875095238, 
  0.8819571429], [0.0059571429, 0.4086142857, 0.8828428571], 
 [0.0165142857, 0.4266, 0.8786333333], [0.032852381, 0.4430428571, 
  0.8719571429], [0.0498142857, 0.4585714286, 0.8640571429], 
 [0.0629333333, 0.4736904762, 0.8554380952], [0.0722666667, 0.4886666667, 
  0.8467], [0.0779428571, 0.5039857143, 0.8383714286], 
 [0.079347619, 0.5200238095, 0.8311809524], [0.0749428571, 0.5375428571, 
  0.8262714286], [0.0640571429, 0.5569857143, 0.8239571429], 
 [0.0487714286, 0.5772238095, 0.8228285714], [0.0343428571, 0.5965809524, 
  0.819852381], [0.0265, 0.6137, 0.8135], [0.0238904762, 0.6286619048, 
  0.8037619048], [0.0230904762, 0.6417857143, 0.7912666667], 
 [0.0227714286, 0.6534857143, 0.7767571429], [0.0266619048, 0.6641952381, 
  0.7607190476], [0.0383714286, 0.6742714286, 0.743552381], 
 [0.0589714286, 0.6837571429, 0.7253857143], 
 [0.0843, 0.6928333333, 0.7061666667], [0.1132952381, 0.7015, 0.6858571429], 
 [0.1452714286, 0.7097571429, 0.6646285714], [0.1801333333, 0.7176571429, 
  0.6424333333], [0.2178285714, 0.7250428571, 0.6192619048], 
 [0.2586428571, 0.7317142857, 0.5954285714], [0.3021714286, 0.7376047619, 
  0.5711857143], [0.3481666667, 0.7424333333, 0.5472666667], 
 [0.3952571429, 0.7459, 0.5244428571], [0.4420095238, 0.7480809524, 
  0.5033142857], [0.4871238095, 0.7490619048, 0.4839761905], 
 [0.5300285714, 0.7491142857, 0.4661142857], [0.5708571429, 0.7485190476, 
  0.4493904762], [0.609852381, 0.7473142857, 0.4336857143], 
 [0.6473, 0.7456, 0.4188], [0.6834190476, 0.7434761905, 0.4044333333], 
 [0.7184095238, 0.7411333333, 0.3904761905], 
 [0.7524857143, 0.7384, 0.3768142857], [0.7858428571, 0.7355666667, 
  0.3632714286], [0.8185047619, 0.7327333333, 0.3497904762], 
 [0.8506571429, 0.7299, 0.3360285714], [0.8824333333, 0.7274333333, 0.3217], 
 [0.9139333333, 0.7257857143, 0.3062761905], [0.9449571429, 0.7261142857, 
  0.2886428571], [0.9738952381, 0.7313952381, 0.266647619], 
 [0.9937714286, 0.7454571429, 0.240347619], [0.9990428571, 0.7653142857, 
  0.2164142857], [0.9955333333, 0.7860571429, 0.196652381], 
 [0.988, 0.8066, 0.1793666667], [0.9788571429, 0.8271428571, 0.1633142857], 
 [0.9697, 0.8481380952, 0.147452381], [0.9625857143, 0.8705142857, 0.1309], 
 [0.9588714286, 0.8949, 0.1132428571], [0.9598238095, 0.9218333333, 
  0.0948380952], [0.9661, 0.9514428571, 0.0755333333], 
 [0.9763, 0.9831, 0.0538]]

parula_map = LinearSegmentedColormap.from_list('parula', cm_data)